## Measure the Energy Intensity of Running Tabular DDPMs ##

Things we care about:
1) Model Size
2) Model Type (CNN or MLP)
3) Dataset size
4) Batch Size
5) Data Type (Image or Tabular)
6) Training time
7) Sampling time
8) Sampled Data size

We hope to use the experiment run with this script to evaluate the carbon intensity of DDPMs

Training: Run DDPM training until convergence

Sampling: Generate 8192 samples with batch size parameter

In [1]:
#import the goods

import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import TabDiffusionTools as TDT
import time

"""
start = time.time()

-Run

end = time.time()

clock = end - start
"""

ModuleNotFoundError: No module named 'TabDiffusionTools'

In [ ]:
#Set up the loop:
machine_id = 'Quadro_RTX_4000_Shannon'
DS_sizes = [1024,2048, 4096]
batch_size = 512
patience = 512

Net_sizes = [[32,32],
             [64,64],
             [128,128,128,128],
             [256,256,256,256]]

t_dim = 32


# Load in parametric wave data
Wave_4 = np.concatenate((np.load('./dataset/Wave_4_Img_64/params.npy'), np.load('./dataset/Wave_4_Img_128/params.npy')), axis = 0)

print(Wave_4.shape) # should be (8192,2,5)

Wave_4 = Wave_4.reshape((4096,20))
print(Wave_4.shape) # should be (8192,10)

Wave_8 = np.concatenate((np.load('./dataset/Wave_8_Img_64/params.npy'), np.load('./dataset/Wave_8_Img_128/params.npy')), axis = 0)

print(Wave_8.shape) # should be (8192,8,5)


Wave_8 = Wave_8.reshape((4096,40))

print(Wave_8.shape) # should be (8192,40)

Wave = [Wave_4, Wave_8]

val_limits = [[0,255],
                      [0.5,8],
                      [-np.pi,np.pi],
                      [-np.pi,np.pi],
                      [0,255]]





(4096, 4, 5)
(4096, 20)
(4096, 8, 5)
(4096, 40)


In [ ]:
labels = ['run_id', 'model size', 'training epochs', 'train time', 'sample time 8192 samples']
Results = []
Results.append(labels)


for i in range(len(Wave)):
    for j in  range(len(DS_sizes)):
     # Set up the training data:
        DS = Wave[i][0:DS_sizes[j]]
        num_waves = int(DS.shape[1]//5)
        print(DS.shape)
        LIMITS = []
        for k in range(DS.shape[1]):
            LIMITS.append(val_limits[k%5])
        LIMITS = np.array(LIMITS)

        for k in range(len(Net_sizes)):
            DDPM_Dict = {
                        'xdim' : DS.shape[1],
                        'X_LL' : LIMITS[:,0],
                        'X_UL' : LIMITS[:,1],
                        'ydim': 0,
                        'cdim': 0,
                        'datalength': len(DS),
                        'tdim': t_dim,
                        'net': Net_sizes[k],
                        'batch_size': batch_size,
                        'Training_Epochs': 100000,
                        'Diffusion_Timesteps': 128,
                        'lr' : 0.00025,                 # learning rate
                        'weight_decay': 0.0,            # weight decay
                        'device_name': 'cuda:0'}        # gpu device name}

            # Set up the model

            T = TDT.DiffusionEnv(DDPM_Dict,
                X=DS)
                #Y=Y_set,
                #Cons = Con_set,
                #dataLength=num_samples,
                #lr = 0.00025,
                #weight_decay=0.0,
                #device=torch.device('cuda:0'))
            model_size = 0
            for p in T.diffusion.parameters():
                model_size += p.numel()

            # Train the model
            start = time.time()

            epochs = T.run_train_loop_patience(batches_per_epoch=100, patience = 1000)

            end = time.time()
            clock_train = end - start

            # Sample the model
            start = time.time()
            X_gen, unnorm = T.gen_samples(8192)
            end = time.time()

            clock_sample = end - start
            id = 'Tab_Wave_' + str(num_waves) + '_DS_' + str(DS_sizes[j]) + '_Net_' + str(k)
            np.save('./generated_samples/Shannon_RTX_4000/' + id + '_Samples_' + machine_id + '.npy', X_gen)

            Results.append([id,model_size, epochs, clock_train, clock_sample])



(1024, 20)
Step 500/100000 Loss: 0.34430384635925293
Step 1000/100000 Loss: 0.2991546094417572
Step 1500/100000 Loss: 0.30185145139694214
Step 2000/100000 Loss: 0.27196940779685974
Step 2500/100000 Loss: 0.31362801790237427
Step 3000/100000 Loss: 0.2727885842323303
Step 3500/100000 Loss: 0.263915479183197
Step 4000/100000 Loss: 0.26723822951316833
Step 4500/100000 Loss: 0.27648159861564636


100%|██████████| 127/127 [00:00<00:00, 1352.58it/s]


Step 500/100000 Loss: 0.32534095644950867
Step 1000/100000 Loss: 0.28966769576072693
Step 1500/100000 Loss: 0.2652687132358551
Step 2000/100000 Loss: 0.24309611320495605
Step 2500/100000 Loss: 0.25609269738197327
Step 3000/100000 Loss: 0.2498127520084381
Step 3500/100000 Loss: 0.29389163851737976


100%|██████████| 127/127 [00:00<00:00, 1313.14it/s]


Step 500/100000 Loss: 0.31188511848449707
Step 1000/100000 Loss: 0.29368510842323303
Step 1500/100000 Loss: 0.2921791672706604
Step 2000/100000 Loss: 0.2560029923915863
Step 2500/100000 Loss: 0.23677611351013184
Step 3000/100000 Loss: 0.24086351692676544
Step 3500/100000 Loss: 0.24155913293361664
Step 4000/100000 Loss: 0.2526160478591919


...

100%|██████████| 127/127 [00:00<00:00, 1207.54it/s]


Step 500/100000 Loss: 0.44135138392448425
Step 1000/100000 Loss: 0.4332573413848877
Step 1500/100000 Loss: 0.41786810755729675
Step 2000/100000 Loss: 0.417950838804245
Step 2500/100000 Loss: 0.39552003145217896


100%|██████████| 127/127 [00:00<00:00, 927.09it/s]


Step 500/100000 Loss: 0.4296523630619049
Step 1000/100000 Loss: 0.4467632472515106
Step 1500/100000 Loss: 0.42135390639305115
Step 2000/100000 Loss: 0.4151332974433899
Step 2500/100000 Loss: 0.39981818199157715
Step 3000/100000 Loss: 0.4060977101325989
Step 3500/100000 Loss: 0.39773446321487427
Step 4000/100000 Loss: 0.3861753046512604


100%|██████████| 127/127 [00:00<00:00, 482.32it/s]


In [ ]:
import csv

path = './Tab_Wave_Results_' + machine_id +'.csv'

f =  open(path, 'w', newline='')
writer = csv.writer(f)
writer.writerows(Results)
f.close()